In [ ]:
import os
import numpy as np
import pandas as pd
import gc
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout, MaxPool2D, LeakyReLU, Conv2D, Flatten
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from sklearn.model_selection import train_test_split

In [ ]:
input_path = 'C:/SSD_Projects/Bengali_git/input/'
train_df_ = pd.read_csv(os.path.join(input_path, 'train.csv'))
train_df_ = np.array_split(train_df_,4)
img_size = 128

In [ ]:
def read_data(nf):
    nf=int(nf)
    train_df = np.load(os.path.join(input_path,f'train_image_data_{nf}.npy'))
    return train_df
def get_rand_bbox(width, height, l):
    r_x = np.random.randint(width)
    r_y = np.random.randint(height)
    r_l = np.sqrt(1 - l)
    r_w = np.int(width * r_l)
    r_h = np.int(height * r_l)
    return r_x, r_y, r_l, r_w, r_h
class MultiOutputDataGenerator(keras.preprocessing.image.ImageDataGenerator):
    # custom image generator
    def __init__(self, featurewise_center = False, samplewise_center = False, 
                 featurewise_std_normalization = False, samplewise_std_normalization = False, 
                 zca_whitening = False, zca_epsilon = 1e-06, rotation_range = 0.0, width_shift_range = 0.0, 
                 height_shift_range = 0.0, brightness_range = None, shear_range = 0.0, zoom_range = 0.0, 
                 channel_shift_range = 0.0, fill_mode = 'nearest', cval = 0.0, horizontal_flip = False, 
                 vertical_flip = False, rescale = None, preprocessing_function = None, data_format = None, validation_split = 0.0, 
                 mix_up_alpha = 0.0): # additional class argument
    

        super().__init__(featurewise_center, samplewise_center, featurewise_std_normalization, samplewise_std_normalization, 
                         zca_whitening, zca_epsilon, rotation_range, width_shift_range, height_shift_range, brightness_range, 
                         shear_range, zoom_range, channel_shift_range, fill_mode, cval, horizontal_flip, vertical_flip, rescale, 
                         preprocessing_function, data_format, validation_split)

        # Mix-up
        assert mix_up_alpha >= 0.0
        self.mix_up_alpha = mix_up_alpha
        


    def mix_up(self, X1, y1, X2, y2, ordered_outputs, target_lengths):
        assert X1.shape[0] == y1.shape[0] == X2.shape[0] == y2.shape[0]
        batch_size = X1.shape[0]
        l = np.random.beta(self.mix_up_alpha, self.mix_up_alpha, batch_size)
        X_l = l.reshape(batch_size, 1, 1, 1)
        y_l = l.reshape(batch_size, 1)
        X = X1 * X_l + X2 * (1-X_l)
        target_dict = {}
        i = 0
        for output in ordered_outputs:
            target_length = target_lengths[output]
            target_dict[output] = y1[:, i: i + target_length] * y_l + y2[:, i: i + target_length] * (1 - y_l)
            i += target_length
        y = None
        for output, target in target_dict.items():
            if y is None:
                y = target
            else:
                y = np.concatenate((y, target), axis=1)
        return X, y
    
    
    def flow(self,
             x,
             y=None,
             batch_size=32,
             shuffle=True,
             sample_weight=None,
             seed=None,
             save_to_dir=None,
             save_prefix='',
             save_format='png',
             subset=None):
        
        # for multi-outputs
        targets = None
        target_lengths = {}
        ordered_outputs = []
        for output, target in y.items():
            if targets is None:
                targets = target
            else:
                targets = np.concatenate((targets, target), axis=1)
            target_lengths[output] = target.shape[1]
            ordered_outputs.append(output)
        
        # parent flow
        batches = super().flow(x, targets, batch_size, shuffle, sample_weight, seed, save_to_dir, save_prefix, save_format, subset)
        
        # custom processing
        while True:
            batch_x, batch_y = next(batches)
            
            # mixup or cutmix
            if (self.mix_up_alpha > 0):
                while True:
                    batch_x_2, batch_y_2 = next(batches)
                    m1, m2 = batch_x.shape[0], batch_x_2.shape[0]
                    if m1 < m2:
                        batch_x_2 = batch_x_2[:m1]
                        batch_y_2 = batch_y_2[:m1]
                        break
                    elif m1 == m2:
                        break
                if np.random.rand() < 0.5:
                    batch_x, batch_y = self.mix_up(batch_x, batch_y, batch_x_2, batch_y_2, ordered_outputs, target_lengths)
            
                target_dict = {}
                i = 0
                for output in ordered_outputs:
                    target_length = target_lengths[output]
                    target_dict[output] = batch_y[:, i: i + target_length]
                    i += target_length
                    
                yield batch_x, target_dict

In [ ]:
batch_size = 128
momentum = 0.9
dropout_rate = 0.2
alpha = 0.1
epsilon = 0.00001
lr = 0.001
opt = Adam(learning_rate = lr)

def build_model():
    inputs = Input(shape = (img_size, img_size, 1))

    model = Conv2D(filters=32, kernel_size=(3, 3), padding='SAME', input_shape=(img_size, img_size, 1))(inputs)
    model = BatchNormalization(axis=-1, momentum=momentum, epsilon = epsilon, gamma_initializer="uniform")(model)
    model = LeakyReLU(alpha = alpha)(model)
    model = Conv2D(filters=32, kernel_size=(3, 3), padding='SAME')(model)
    model = BatchNormalization(axis=-1, momentum=momentum, epsilon = epsilon, gamma_initializer="uniform")(model)
    model = LeakyReLU(alpha = alpha)(model)
    model = Conv2D(filters=32, kernel_size=(3, 3), padding='SAME')(model)
    model = BatchNormalization(axis=-1, momentum=momentum, epsilon = epsilon, gamma_initializer="uniform")(model)
    model = LeakyReLU(alpha = alpha)(model)
    
    model = MaxPool2D(pool_size=(2, 2))(model)
    model = Dropout(rate = dropout_rate)(model)
    
    model = Conv2D(filters=64, kernel_size=(3, 3), padding='SAME')(model)
    model = BatchNormalization(axis=-1, momentum=momentum, epsilon = epsilon, gamma_initializer="uniform")(model)
    model = LeakyReLU(alpha = alpha)(model)
    model = Conv2D(filters=64, kernel_size=(3, 3), padding='SAME')(model)
    model = BatchNormalization(axis=-1, momentum=momentum, epsilon = epsilon, gamma_initializer="uniform")(model)
    model = LeakyReLU(alpha = alpha)(model)
    model = Conv2D(filters=64, kernel_size=(3, 3), padding='SAME')(model)
    model = BatchNormalization(axis=-1, momentum=momentum, epsilon = epsilon, gamma_initializer="uniform")(model)
    model = LeakyReLU(alpha = alpha)(model)
    
    model = MaxPool2D(pool_size=(2, 2))(model)
    model = Dropout(rate = dropout_rate)(model)
    
    model = Conv2D(filters=128, kernel_size=(3, 3), padding='SAME')(model)
    model = BatchNormalization(axis=-1, momentum=momentum, epsilon = epsilon, gamma_initializer="uniform")(model)
    model = LeakyReLU(alpha = alpha)(model)
    model = Conv2D(filters=128, kernel_size=(3, 3), padding='SAME')(model)
    model = BatchNormalization(axis=-1, momentum=momentum, epsilon = epsilon, gamma_initializer="uniform")(model)
    model = LeakyReLU(alpha = alpha)(model)
    model = Conv2D(filters=128, kernel_size=(3, 3), padding='SAME')(model)
    model = BatchNormalization(axis=-1, momentum=momentum, epsilon = epsilon, gamma_initializer="uniform")(model)
    model = LeakyReLU(alpha = alpha)(model)
    
    model = MaxPool2D(pool_size=(2, 2))(model)
    model = Dropout(rate = dropout_rate)(model)
    
    model = Conv2D(filters=256, kernel_size=(3, 3), padding='SAME')(model)
    model = BatchNormalization(axis=-1, momentum=momentum, epsilon = epsilon, gamma_initializer="uniform")(model)
    model = LeakyReLU(alpha = alpha)(model)
    model = Conv2D(filters=256, kernel_size=(3, 3), padding='SAME')(model)
    model = BatchNormalization(axis=-1, momentum=momentum, epsilon = epsilon, gamma_initializer="uniform")(model)
    model = LeakyReLU(alpha = alpha)(model)
    
    model = MaxPool2D(pool_size=(2, 2))(model)
    model = Dropout(rate = dropout_rate)(model)
    
    model = Conv2D(filters=512, kernel_size=(3, 3), padding='SAME')(model)
    model = BatchNormalization(axis=-1, momentum=momentum, epsilon = epsilon, gamma_initializer="uniform")(model)
    model = LeakyReLU(alpha = alpha)(model)
    model = Conv2D(filters=512, kernel_size=(3, 3), padding='SAME')(model)
    model = BatchNormalization(axis=-1, momentum=momentum, epsilon = epsilon, gamma_initializer="uniform")(model)
    model = LeakyReLU(alpha = alpha)(model)
    
    model = MaxPool2D(pool_size=(2, 2))(model)
    model = Dropout(rate = dropout_rate)(model)
    
    model = Flatten()(model)
    model = Dense(512)(model)
    model = LeakyReLU(alpha = alpha)(model)
    dense = BatchNormalization(axis=-1, momentum=momentum)(model)

    head_root = Dense(168, activation = 'softmax')(dense)
    head_vowel = Dense(11, activation = 'softmax')(dense)
    head_consonant = Dense(7, activation = 'softmax')(dense)

    model = tf.keras.models.Model(inputs=inputs, outputs=[head_root, head_vowel, head_consonant])  
    return model
model = build_model()
model.compile(optimizer=opt, loss=CategoricalCrossentropy(label_smoothing=0.1), metrics=['accuracy'])
model.summary()

In [ ]:
learning_rate_reduction_root = ReduceLROnPlateau(monitor='dense_1_accuracy',
                                                                    patience=3,
                                                                    verbose=1,
                                                                    factor=0.5,
                                                                    min_lr=0.00001)
learning_rate_reduction_vowel = ReduceLROnPlateau(monitor='dense_2_accuracy',
                                                                    patience=3,
                                                                    verbose=1,
                                                                    factor=0.5,
                                                                    min_lr=0.00001)
learning_rate_reduction_consonant = ReduceLROnPlateau(monitor='dense_3_accuracy',
                                                                    patience=3,
                                                                    verbose=1,
                                                                    factor=0.5,
                                                                    min_lr=0.00001)

In [ ]:
for i in range(4):
    x_df = read_data(i)

    y_df_root = pd.get_dummies(train_df_[i]['grapheme_root']).values
    y_df_vowel = pd.get_dummies(train_df_[i]['vowel_diacritic']).values
    y_df_consonant = pd.get_dummies(train_df_[i]['consonant_diacritic']).values

    x_train, x_test, y_train_root, y_test_root, y_train_vowel, y_test_vowel, y_train_consonant, y_test_consonant = train_test_split(x_df, y_df_root, y_df_vowel, y_df_consonant, test_size=0.01, random_state = 111)

    del x_df
    del y_df_root, y_df_vowel, y_df_consonant

    datagen = MultiOutputDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=5,  # randomly rotate images in the range (degrees, 0 to 180, was 8)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False,
        mix_up_alpha = 0.4)
        #cutmix_alpha = 0.4) 

    datagen.fit(x_train)

    model.fit_generator(datagen.flow(x_train, {'dense_1': y_train_root, 'dense_2': y_train_vowel, 'dense_3': y_train_consonant}, batch_size=batch_size),
                        epochs = 15, 
                        validation_data = (x_test, [y_test_root, y_test_vowel, y_test_consonant]), 
                        callbacks = [learning_rate_reduction_root,learning_rate_reduction_vowel, learning_rate_reduction_consonant],
                        steps_per_epoch = len(x_train) // batch_size
                       )

    del x_train, x_test
    del y_train_root, y_test_root
    del y_train_vowel, y_test_vowel
    del y_train_consonant, y_test_consonant
    gc.collect()
    

In [ ]:
model.save('model.h5')